# Universidad Distrital Francisco José de Caldas
## Class: Advanced Image Processing - Remote Sensing with Python (Basic Activities)
### Por: David Alonso Rueda Rodríguez

### 2. Crear Imágenes Stack

#### Objetivo de la actividad:
- Crear layer stacked para la imagen Sentinel RGB+NIR.

#### Librerías requeridas:
- **rasterio** (librería para comparación)
- **gdal** (librería para comparación)
- **earthpy** (librería para comparación)
- datetime

#### Fuente de datos:

https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

In [34]:
import rasterio as ras
import glob
import json
import os
import re
import earthpy as et
import earthpy.spatial as es
from osgeo import gdal
from datetime import datetime

In [5]:
def time():
    from datetime import datetime as dt
    now = dt.now()
    timestamp = dt.timestamp(now)
    return timestamp

## Preparación de datos de prueba

### Determinación de Resolución espacial para bandas

Es necesario hacer una identificación de la resolución espacial y con este dato hacer la concatenacióno de las imágenes (layer stack), en caso de realizar la concatenación con todas las bandas estás serán remuestreadas a la menor resolución espacial disponible.

### Imágenes Sentinel
#### Obtener datos desde directorio de trabajo

In [27]:
# Obtener archivos jp2 en el directorio de interés
url_base = 'src/Sentinel/'
images = glob.glob(url_base + '*.jp2')
images

['src/Sentinel/T18NVL_20200104T152639_B01.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B02.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B03.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B04.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B05.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B06.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B07.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B08.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B08A.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B09.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B10.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B11.jp2']

#### Identificación de resolución espacial para las bandas de la imagen satelital

In [26]:
images[0]
a = ras.open(images[0])

In [30]:
# Resolución espacial de las bandas en la imagen satelital
bandas = []
res = []
print('Nombre Arhivo'+ '\t'*3 + 'Res. Espacial')
for image in images:
    with ras.open(image) as src:
        a1 = os.path.splitext(os.path.basename(image))[0]
        print('{}\t\t{:.0f}' .format(a1, src.profile['transform'][0], (-1)*src.profile['transform'][4]))
        bandas.append(a1)
        res.append(str(src.profile['transform'][0]) + ', ' + str(src.profile['transform'][2]))

Nombre Arhivo			Res. Espacial
LC08_L1TP_009057_20190903_20190917_01_T1_B2		30
LC08_L1TP_009057_20190903_20190917_01_T1_B3		30
LC08_L1TP_009057_20190903_20190917_01_T1_B4		30
LC08_L1TP_009057_20190903_20190917_01_T1_B5		30


### Resumen de datos para trabajarse

#### - 10 metros resolución espacial

- T18NVL_20200104T152639_B02: 10
- T18NVL_20200104T152639_B03: 10
- T18NVL_20200104T152639_B04: 10
- T18NVL_20200104T152639_B08: 10

#### - 20 metros resolución espacial

- T18NVL_20200104T152639_B05: 20
- T18NVL_20200104T152639_B06: 20
- T18NVL_20200104T152639_B07: 20
- T18NVL_20200104T152639_B08A: 20
- T18NVL_20200104T152639_B11: 20
- T18NVL_20200104T152639_B12: 20

#### - 60 metros resolución espacial

- T18NVL_20200104T152639_B01: 60
- T18NVL_20200104T152639_B09: 60
- T18NVL_20200104T152639_B10: 60

In [16]:
# Discriminación de datos por resolución espacial
sentinel_10 = [
    'T18NVL_20200104T152639_B02.jp2', 'T18NVL_20200104T152639_B03.jp2',
    'T18NVL_20200104T152639_B04.jp2', 'T18NVL_20200104T152639_B08.jp2'
]
sentinel_20 = [
    'T18NVL_20200104T152639_B05.jp2', 'T18NVL_20200104T152639_B06.jp2', 
    'T18NVL_20200104T152639_B07.jp2', 'T18NVL_20200104T152639_B08A.jp2', 
    'T18NVL_20200104T152639_B11.jp2', 'T18NVL_20200104T152639_B12.jp2'
]
sentinel_60 = [
    'T18NVL_20200104T152639_B01.jp2', 'T18NVL_20200104T152639_B09.jp2',
    'T18NVL_20200104T152639_B10.jp2'
]

In [17]:
# Agregación de url_base con listado de archivos
sentinel_10m = []
sentinel_20m = []
for i in sentinel_10:
    sentinel_10m.append(url_base + i)
for i in sentinel_20:
    sentinel_20m.append(url_base + i)  

### Imágenes Sentinel
#### Obtener datos desde directorio de trabajo

In [35]:
# Obtener archivos jp2 en el directorio de interés
url_base = 'src/Sentinel/'
images = glob.glob(url_base + '*.jp2')
images

['src/Sentinel/T18NVL_20200104T152639_B01.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B02.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B03.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B04.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B05.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B06.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B07.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B08.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B08A.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B09.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B10.jp2',
 'src/Sentinel/T18NVL_20200104T152639_B11.jp2']

In [36]:
# Resolución espacial de las bandas en la imagen satelital
bandas = []
res = []
print('Nombre Arhivo'+ '\t'*3 + 'Res. Espacial')
for image in images:
    with ras.open(image) as src:
        a1 = os.path.splitext(os.path.basename(image))[0]
        print('{}\t\t{:.0f}' .format(a1, src.profile['transform'][0], (-1)*src.profile['transform'][4]))
        bandas.append(a1)
        res.append(str(src.profile['transform'][0]) + ', ' + str(src.profile['transform'][2]))

Nombre Arhivo			Res. Espacial


RasterioIOError: ../../../../../../Datos/Imagenes/Sentinel/T18NVL_20200104T152639_B01.jp2: No such file or directory